%%HTML
<style>

.CodeMirror {
    width: 100vw;
}

.container {
    width: 99% !important;
}

.rendered_html {
  font-size:0.7em;
}
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
     font-size: 100%;
}

</style>

## Flashing device

# edc2018micropython

https://micropython.org/download#esp32

`pip install esptool`

### USB name
- `/dev/ttyUSB0` (Linux)

## ESP32

- Internal 4 MB RAM?

ESP32 can use external SPIRAM (psRAM) to expand available RAM up to 16MB [source](https://github.com/loboris/MicroPython_ESP32_psRAM_LoBo).

In [2]:
!esptool.py --port /dev/ttyUSB0 --chip esp32 erase_flash

esptool.py v2.4.0
Connecting....
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse
MAC: 80:7d:3a:c5:3a:34
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 6.1s
Hard resetting via RTS pin...


In [3]:
!esptool.py --chip esp32 --port /dev/ttyUSB0 write_flash -z 0x1000 esp32-20180925-v1.9.4-575-g6ea6c7cc9.bin

esptool.py v2.4.0
Connecting....
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse
MAC: 80:7d:3a:c5:3a:34
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1053296 bytes to 663038...
Wrote 1053296 bytes (663038 compressed) at 0x00001000 in 58.8 seconds (effective 143.2 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


# Communication

### UART
- "universal asynchronous receiver-transmitter"
- **PC -> (USB) -> ESP32 **
- serial - parallel (1 bit <-> bytes)
- baudrate (bits per second) (default 115200)
- Tools
  - screen
  - Putty
  - Picocom
  - YAT (Yet Another Terminal)
  - pySerial
  - Jupyter Notebook
  

## Bind tty to static name (Linux)

/etc/udev/rules.d/10-local.rules

`ACTION=="add", ATTRS{idVendor}=="0403", ATTRS{idProduct}=="6001", SYMLINK+="my_uart"`

You can check for the variables of your device by running

`udevadm info --name=/dev/ttyUSBx --attribute-walk`

 ATTRS{idProduct}=="0002"
 ATTRS{idVendor}=="1d6b"

## File transfer

### ampy

`pip install adafruit-ampy`

`ampy -p /dev/ttyUSB0 put boot.py`

### FTP

google `ftp server micropython`

In [ ]:
!ampy -p /dev/micropython ls

https://github.com/SebastianRoll/slides_micropython

## WebREPL


![webrepl](https://cdn-learn.adafruit.com/assets/assets/000/035/130/original/microcontrollers_Screen_Shot_2016-08-25_at_6.41.48_PM.png?1472175855)



http://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html

`import webrepl_setup`

- uses a single connection over WebSocket
- `wlan.config('mac')`

If the board isn't connected already to a wireless access point then it will create one which you can connect to from your computer.

## REPL (read-eval-print-loop)

- help(), help('modules'), dir()
- auto-indent, auto-completion
- Interrupting
- paste mode
- bitbucket variable

In [7]:
help('modules')


__main__          framebuf          socket            upip
_boot             gc                ssl               upip_utarfile
_onewire          hashlib           struct            upysh
_thread           heapq             sys               urandom
_webrepl          inisetup          time              ure
apa106            io                ubinascii         urequests
array             json              ucollections      uselect
binascii          machine           ucryptolib        usocket
btree             math              uctypes           ussl
builtins          micropython       uerrno            ustruct
cmath             neopixel          uhashlib          utime
collections       network           uhashlib          utimeq
dht               ntptime           uheapq            uzlib
ds18x20           onewire           uio               webrepl
errno             os                ujson             webrepl_setup
esp               random            umqtt/robust      websocket
esp32    

In [8]:
import esp
print(esp.flash_size())

4194304


## Reset
***
__soft__ `CTRL+D`
- clears the state of the MicroPython virtual machine
- leaves hardware peripherals unaffected
  
```
import sys
sys.exit()
```
***
**hard**

- same as performing a power cycle to the boar

```
import machine
machine.reset()
```

#### Startup code

- boot.py: power cycle, hard reset, deep sleep
- main.py: your main program

In [9]:
with open('boot.py', 'r') as f:
    print(f.read())

# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
#import webrepl
#webrepl.start()



## Machine module

In [ ]:
import machine
print(help(machine))

### RTC (real time clock)

In [ ]:
rtc = machine.RTC()
rtc.init((2014, 5, 1, 4, 13, 0, 0, 0))
print(rtc.datetime())

### Power control

In [ ]:
import machine

rtc = machine.RTC()
rtc.irq(trigger=rtc.ALARM0, wake=machine.DEEPSLEEP)
rtc.alarm(rtc.ALARM0, 10000)
machine.deepsleep()
if machine.reset_cause() == machine.DEEPSLEEP_RESET:
    print("WOKE")
else:
    machine.reset()

#### WiFi

In [ ]:
# Access Point
import network
ap_if = network.WLAN(network.AP_IF)
ap_if.active()
# import webrepl_setup

In [ ]:
# Connect to network
import network
sta_if = network.WLAN(network.STA_IF)
print('connecting to network...')
sta_if.active(True)
sta_if.connect('Clarion-Connect', '')
while not sta_if.isconnected():
    pass
print('network config:', sta_if.ifconfig())

git@github.com:SebastianRoll/NDC-MicroPython.git
`/final`

https://github.com/SebastianRoll/micropython-EDC2018

#### Box!
- mp3 module _YX5300_
- OLED display _SSD-1351_
- microphone _Max7219(?)_
- MicroSD module
- pushbuttons


### upip (package manager)

In [ ]:
import upip
upip.install('micropython-collections')

### micropython-lib
Core Python libraries ported to MicroPython

https://github.com/micropython/micropython-lib